In [51]:
import os 
import sys
import numpy as np
import tensorflow as tf

from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [3]:
data_path = "./dataset/"
values_file = "./values"

spp_files = ["".join((data_path, f)) for f in os.listdir(data_path) if os.path.isfile("".join((data_path, f)))]
spp_files = filter(lambda x: x.split(".")[-1] == "spp", spp_files)

values_data = []
with open(values_file, "r") as f:
    f.readline()
    for line in f:
        tmp = line.strip(" ,\n").split('\t')
        tmp = [int(tmp[0]), float(tmp[1])]
        values_data.append(tmp)
        
values_indexes = [i[0] for i in values_data]

processed_indexes = []
for spf in spp_files:
    ind = spf.split('/')[-1]
    ind = ind.split('.')[0]
    
    processed_indexes.append(int(ind))
    
processed_indexes = sorted(processed_indexes)

remaped_indexes = dict()
for ind in xrange(0, len(processed_indexes)):
    remaped_indexes[processed_indexes[ind]] = ind 
    
remaped_keys = set(remaped_indexes.keys())
values = [0.0]*len(processed_indexes)
for val in values_data:
    if val[0] in remaped_keys:
        values[remaped_indexes[val[0]]] = val[1]
        
molecul_data = dict()

for spf in spp_files:
    ind = spf.split('/')[-1]
    ind = ind.split('.')[0]
    
    molecul_data[int(ind)] = list()
    
    with open(spf, "r") as f:
        for line in f:
            tmp = line.strip(" \n").split(",")
            tmp = [float(i.strip()) for i in tmp]
            molecul_data[int(ind)].append(tmp)
            

sorted_data = sorted(zip(molecul_data.keys(), molecul_data.values()), key = lambda x: x[0])

dataset = []
for i in sorted_data:
    dataset.append(i[1])
    
dataset = np.array(dataset)
print dataset.shape

### Подготовка данных для автоэнкодера

In [14]:
data_to_encode = []
for mol in dataset:
    for sp in mol:
        data_to_encode.append(sp)

In [15]:
data_to_encode_split = np.copy(data_to_encode)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(data_to_encode_split, data_to_encode_split, test_size=0.33, random_state=42)

In [17]:
unencoded_scaler = StandardScaler()
x_train_scaled = unencoded_scaler.fit_transform(x_train)
x_test_scaled = unencoded_scaler.transform(x_test)

In [18]:
x_data = unencoded_scaler.transform(data_to_encode)

In [19]:
x_train_scaled = np.array(x_train_scaled)
x_test_scaled  = np.array(x_test_scaled)

In [20]:
h0_num = x_test_scaled.shape[1]
h1_num = 80
h2_num = 50
h3_num = 8

In [21]:
input_img = keras.layers.Input(shape=(h0_num,))
input_encoded = keras.layers.Input(shape=(h3_num,))

x = keras.layers.Dense(h0_num, activation='elu')(input_img)
x = keras.layers.Dense(h1_num, activation='elu')(x)
x = keras.layers.Dense(h2_num, activation='elu')(x)
encoded = keras.layers.Dense(h3_num, activation='elu')(x)

x = keras.layers.Dense(h3_num, activation='elu')(input_encoded)
x = keras.layers.Dense(h2_num, activation='elu')(x)
x = keras.layers.Dense(h1_num, activation='elu')(x)
decoded = keras.layers.Dense(h0_num, activation='elu')(x)

In [22]:
encoder = keras.models.Model(input_img, encoded, name="encoder")
decoder = keras.models.Model(input_encoded, decoded, name="decoder")

autoencoder = keras.models.Model(input_img, decoder(encoder(input_img)))
autoencoder.compile(optimizer='RMSprop', loss='mean_squared_error')

In [23]:
autoencoder.fit(x_train_scaled, x_train_scaled, epochs=100, batch_size=256, shuffle=False, validation_data=(x_test_scaled, x_test_scaled))

Train on 467233 samples, validate on 230130 samples
Epoch 1/100
467233/467233 [==============================]467233/467233 [==============================] - 3s 6us/step - loss: 0.2606 - val_loss: 0.1825

Epoch 2/100
467233/467233 [==============================]467233/467233 [==============================] - 3s 6us/step - loss: 0.1630 - val_loss: 0.1617

Epoch 3/100
467233/467233 [==============================]467233/467233 [==============================] - 3s 6us/step - loss: 0.1498 - val_loss: 0.1539

Epoch 4/100
467233/467233 [==============================]467233/467233 [==============================] - 3s 6us/step - loss: 0.1418 - val_loss: 0.1474

Epoch 5/100
467233/467233 [==============================]467233/467233 [==============================] - 3s 6us/step - loss: 0.1365 - val_loss: 0.1429

Epoch 6/100
467233/467233 [==============================]467233/467233 [==============================] - 3s 6us/step - loss: 0.1329 - val_loss: 0.1392

Epoch 7/100
467233/46723

467233/467233 [==============================]467233/467233 [==============================] - 3s 6us/step - loss: 0.0998 - val_loss: 0.1055

Epoch 54/100
467233/467233 [==============================]467233/467233 [==============================] - 3s 6us/step - loss: 0.0997 - val_loss: 0.1043

Epoch 55/100
467233/467233 [==============================]467233/467233 [==============================] - 3s 6us/step - loss: 0.0995 - val_loss: 0.1037

Epoch 56/100
467233/467233 [==============================]467233/467233 [==============================] - 3s 6us/step - loss: 0.0994 - val_loss: 0.1049

Epoch 57/100
467233/467233 [==============================]467233/467233 [==============================] - 3s 6us/step - loss: 0.0992 - val_loss: 0.1023

Epoch 58/100
467233/467233 [==============================]467233/467233 [==============================] - 3s 6us/step - loss: 0.0991 - val_loss: 0.1055

Epoch 59/100
467233/467233 [==============================]467233/467233 [=========

In [24]:
x_data_encoded = encoder.predict(x_train_scaled)

### Кластеризация данных

In [25]:
for d in np.linspace(0.4,0.8,4):
    for i in xrange(15, 25, 5):
        print "{} {}".format(i, d)
        db_scan = DBSCAN(min_samples=i, eps=d, algorithm='kd_tree').fit(x_data_encoded)
        labels = db_scan.labels_
        unique_labels = set(labels)
        
        for i in unique_labels:
            print "{} : {}".format(i, float(len([l for l in labels if l == i])) / len(labels))

15 0.4
0 : 0.0674674091941
1 : 0.000149818184931
2 : 0.0389334657441
3 : 0.00226011433268
4 : 0.000190483120841
5 : 0.0234914913972
6 : 0.00146393769276
7 : 0.00011771428816
8 : 0.00643790143248
9 : 3.2103896771e-05
10 : 5.5646754403e-05
11 : 0.000669901312621
12 : 0.00418848839872
13 : 0.000714846768101
14 : 0.0056353040132
15 : 0.000220446757827
16 : 7.49090924656e-05
17 : 0.00312477928571
18 : 0.000695584430038
19 : 0.00610830142563
20 : 0.0063779741585
21 : 0.000263251953522
22 : 0.000312477928571
23 : 0.00361275851663
24 : 0.000785475340997
25 : 0.000355283124266
26 : 0.000102732469667
27 : 0.000224727277397
28 : 0.000624955857142
29 : 0.000109153249021
30 : 0.00052436364726
31 : 0.000351002604696
32 : 0.000691303910469
33 : 0.000393807800391
34 : 0.00189412990949
35 : 6.20675337572e-05
36 : 5.5646754403e-05
37 : 0.000136976626223
38 : 0.000363844163405
39 : 0.000727688326809
40 : 0.000173361042563
41 : 0.000528644166829
42 : 0.000254690914383
43 : 9.84519500977e-05
44 : 0.0003617

352 : 3.2103896771e-05
353 : 6.63480533267e-05
354 : 3.63844163405e-05
355 : 2.99636369863e-05
356 : 3.42441565557e-05
357 : 3.2103896771e-05
358 : 4.49454554794e-05
359 : 6.20675337572e-05
360 : 2.99636369863e-05
361 : 5.13662348336e-05
362 : 3.2103896771e-05
363 : 3.63844163405e-05
364 : 4.70857152641e-05
365 : 4.70857152641e-05
366 : 3.2103896771e-05
367 : 5.13662348336e-05
368 : 3.2103896771e-05
369 : 6.20675337572e-05
370 : 4.49454554794e-05
371 : 3.85246761252e-05
372 : 6.20675337572e-05
373 : 3.2103896771e-05
374 : 4.49454554794e-05
375 : 5.5646754403e-05
376 : 2.56831174168e-05
377 : 3.63844163405e-05
378 : 3.2103896771e-05
379 : 4.70857152641e-05
380 : 2.35428576321e-05
381 : 4.49454554794e-05
382 : 3.42441565557e-05
383 : 5.99272739725e-05
384 : 6.4207793542e-05
385 : 3.2103896771e-05
386 : 3.2103896771e-05
387 : 2.56831174168e-05
388 : 1.92623380626e-05
389 : 3.2103896771e-05
390 : 4.06649359099e-05
391 : 2.14025978473e-05
392 : 3.2103896771e-05
393 : 3.2103896771e-05
394 : 

238 : 3.85246761252e-05
239 : 3.2103896771e-05
240 : 4.28051956947e-05
241 : 4.28051956947e-05
242 : 4.06649359099e-05
243 : 4.49454554794e-05
244 : 4.28051956947e-05
245 : 5.99272739725e-05
246 : 5.77870141878e-05
247 : 4.28051956947e-05
248 : 3.85246761252e-05
249 : 4.28051956947e-05
250 : 4.92259750489e-05
251 : 7.70493522504e-05
252 : 4.28051956947e-05
-1 : 0.823167456066
15 0.533333333333
0 : 0.202860243176
1 : 0.102216667059
2 : 0.00278875849951
3 : 0.00762360535322
4 : 0.0113497976384
5 : 0.0101683742373
6 : 0.030811179861
7 : 0.000177641562133
8 : 0.0024677195318
9 : 0.0450075230131
10 : 0.00901263395351
11 : 0.00424841567269
12 : 0.000132696106653
13 : 0.00509381828766
14 : 0.00678034299803
15 : 0.00442177671526
16 : 0.000194763640411
17 : 0.00258115330039
18 : 0.00716987027885
19 : 0.000139116886008
20 : 0.00167582341145
21 : 0.00116216106311
22 : 0.000284654551369
23 : 0.00247628057094
24 : 0.00130769872847
25 : 0.00067846235176
26 : 0.00263465979501
27 : 4.70857152641e-05
2

336 : 5.13662348336e-05
337 : 2.35428576321e-05
338 : 1.49818184931e-05
339 : 3.42441565557e-05
340 : 5.35064946183e-05
341 : 1.28415587084e-05
342 : 4.92259750489e-05
343 : 6.4207793542e-05
344 : 3.2103896771e-05
345 : 4.92259750489e-05
346 : 2.56831174168e-05
347 : 2.14025978473e-05
348 : 2.35428576321e-05
349 : 5.35064946183e-05
350 : 2.56831174168e-05
351 : 4.28051956947e-05
352 : 3.2103896771e-05
353 : 4.28051956947e-05
354 : 3.2103896771e-05
355 : 3.42441565557e-05
356 : 4.92259750489e-05
357 : 3.63844163405e-05
358 : 3.2103896771e-05
359 : 3.42441565557e-05
360 : 3.2103896771e-05
361 : 4.49454554794e-05
362 : 2.35428576321e-05
363 : 5.13662348336e-05
364 : 3.2103896771e-05
365 : 2.35428576321e-05
366 : 2.56831174168e-05
367 : 2.35428576321e-05
368 : 4.28051956947e-05
369 : 4.70857152641e-05
370 : 2.14025978473e-05
371 : 4.49454554794e-05
372 : 2.35428576321e-05
373 : 4.92259750489e-05
374 : 4.06649359099e-05
375 : 3.2103896771e-05
376 : 2.99636369863e-05
377 : 2.35428576321e-05


238 : 3.42441565557e-05
239 : 4.28051956947e-05
240 : 4.28051956947e-05
241 : 4.92259750489e-05
242 : 4.49454554794e-05
243 : 4.49454554794e-05
244 : 5.99272739725e-05
245 : 3.42441565557e-05
246 : 4.28051956947e-05
247 : 4.49454554794e-05
248 : 4.28051956947e-05
249 : 5.77870141878e-05
250 : 5.5646754403e-05
251 : 3.42441565557e-05
252 : 4.06649359099e-05
253 : 4.49454554794e-05
254 : 4.49454554794e-05
255 : 4.28051956947e-05
256 : 4.28051956947e-05
257 : 4.49454554794e-05
258 : 4.49454554794e-05
259 : 2.99636369863e-05
260 : 2.35428576321e-05
261 : 4.49454554794e-05
262 : 3.2103896771e-05
263 : 3.63844163405e-05
264 : 4.06649359099e-05
265 : 3.63844163405e-05
266 : 4.06649359099e-05
267 : 4.28051956947e-05
268 : 4.49454554794e-05
269 : 4.28051956947e-05
270 : 3.85246761252e-05
271 : 4.06649359099e-05
272 : 4.28051956947e-05
273 : 3.63844163405e-05
274 : 2.99636369863e-05
275 : 4.49454554794e-05
276 : 4.28051956947e-05
277 : 3.85246761252e-05
278 : 2.78233772015e-05
279 : 3.6384416340

70 : 3.42441565557e-05
71 : 5.99272739725e-05
72 : 9.6311690313e-05
73 : 4.28051956947e-05
74 : 0.000134836366438
75 : 2.14025978473e-05
76 : 3.85246761252e-05
77 : 5.35064946183e-05
78 : 0.000143397405577
79 : 6.20675337572e-05
80 : 4.92259750489e-05
81 : 4.06649359099e-05
82 : 5.35064946183e-05
83 : 0.000130555846869
84 : 5.5646754403e-05
85 : 7.91896120351e-05
86 : 4.49454554794e-05
87 : 4.70857152641e-05
88 : 0.000115574028376
89 : 4.06649359099e-05
90 : 7.49090924656e-05
91 : 3.63844163405e-05
92 : 8.13298718198e-05
93 : 0.000109153249021
94 : 3.85246761252e-05
95 : 4.70857152641e-05
96 : 6.20675337572e-05
97 : 4.06649359099e-05
98 : 8.98909109588e-05
99 : 7.49090924656e-05
100 : 3.63844163405e-05
101 : 3.85246761252e-05
102 : 4.49454554794e-05
103 : 2.78233772015e-05
104 : 4.70857152641e-05
105 : 4.06649359099e-05
106 : 1.92623380626e-05
107 : 3.42441565557e-05
108 : 3.2103896771e-05
109 : 6.4207793542e-05
110 : 7.27688326809e-05
111 : 4.28051956947e-05
112 : 3.63844163405e-05
11

In [26]:
db_scan = DBSCAN(min_samples=20, eps=0.8, algorithm='kd_tree').fit(x_data_encoded)

In [30]:
labels = db_scan.labels_
unique_labels = set(labels)

ratios = []
for i in unique_labels:
    ratios.append((i, float(len([l for l in labels if l == i])) / len(labels)))

ratios = sorted(ratios, reverse=True, key=lambda x: x[1])

In [32]:
sum([ratios[i][1] for i in xrange(0,10)])

0.9966248103194765

In [40]:
train_data_new_x = list()
train_data_new_y = list()

for i in xrange(0, 10):
    if ratios[i][0] == -1:
        continue
        
    label_data_x = [x_data_encoded[k] for k in xrange(0, len(x_data_encoded)) if db_scan.labels_[k] == ratios[i][0]]
    label_data_y = [ratios[i][0]]*len(label_data_x)
    
    train_data_new_x = train_data_new_x + label_data_x
    train_data_new_y = train_data_new_y + label_data_y
    
clf = RandomForestClassifier(n_estimators=500, n_jobs=8)
clf.fit(train_data_new_x,train_data_new_y)

In [45]:
r_labels = [ratios[i][0] for i in xrange(0,10) if ratios[i][0] != -1]

In [47]:
remap_r_ind = dict()
for i in xrange(0, len(r_labels)):
    remap_r_ind[r_labels[i]] = i

In [48]:
x_data_last = []
y_data_last = []

for i in xrange(0, len(dataset)):
    m = dataset[i]
    m_scaled = unencoded_scaler.transform(m)
    m_encoded = encoder.predict(m_scaled)
    
    m_labels = clf.predict(m_encoded)
    m_labels = [remap_r_ind[i] for i in m_labels]
    
    descriptor = [0.0]*len(r_labels)

    for l in m_labels:
        descriptor[l] += 1
    
    x_data_last.append(descriptor)
    y_data_last.append(values_data[i])
    
    


In [49]:
x_train_l, x_test_l, y_train_l, y_test_l = train_test_split(x_data_last, y_data_last, test_size=0.33, random_state=42)

In [52]:
rfr = RandomForestRegressor(n_estimators=1000, n_jobs=8).fit(x_train_l, y_train_l)

In [61]:
rfr.score(x_test_l, y_test_l)

0.0

In [54]:
x_data_last[0]

[4325.0, 1891.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [58]:
x_train_l[3]

[6545.0, 731.0, 256.0, 25.0, 25.0, 44.0, 0.0, 0.0, 0.0]